In [122]:
import pandas as pd
import numpy as np
import re
import sklearn.metrics.pairwise as pw
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances

In [123]:
movies = pd.read_csv('./data/movies.csv')
ratings = pd.read_csv('./data/ratings.csv')

#Importing movies rated by my friends
movies_rated = pd.read_excel("./data/bbianchi.xlsx",index_col='Movies')

In [124]:
# Removing duplicated ids with low review count from movie database
movies = movies.loc[~movies['movieId'].isin(duplicated_ids)]
# Removing duplicated ids with low review count from rating database
ratings = ratings.loc[~ratings['movieId'].isin(duplicated_ids)]

movies.head(5)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [125]:

ratings.head(5)

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [126]:
ratings.__class__.__name__

'DataFrame'

In [127]:
#Droping genres 
ratings.drop(['timestamp'], 1, inplace=True) 

In [128]:
df = pd.merge(ratings, movies, on='movieId')
print(df.shape)
# df.head(2)

(100830, 5)


In [129]:
def item_based_recom(input_dataframe,input_film_name):    
    pivot_item_based = pd.pivot_table(input_dataframe,
                                      index='title',
                                      columns=['userId'], values='rating')  
    sparse_pivot = sparse.csr_matrix(pivot_item_based.fillna(0))
    recommender = pw.cosine_similarity(sparse_pivot)
    recommender_df = pd.DataFrame(recommender, 
                                  columns=pivot_item_based.index,
                                  index=pivot_item_based.index)
    ## Item Rating Based Cosine Similarity
    cosine_df = pd.DataFrame(recommender_df[film_name].sort_values(ascending=False))
    cosine_df.reset_index(level=0, inplace=True)
    cosine_df.columns = ['title','cosine_sim']
    return cosine_df

In [130]:
def user_based_recom(input_dataframe,input_user_id):    
    pivot_user_based = pd.pivot_table(input_dataframe, index='title', columns=['userId'], values='rating').T
    sparse_pivot_ub = sparse.csr_matrix(pivot_user_based.fillna(0))
    user_recomm = pw.cosine_similarity(sparse_pivot_ub)
    user_recomm_df = pd.DataFrame(user_recomm,columns=pivot_user_based.index.values,
                 index=pivot_user_based.index.values)
    ## Item Rating Based Cosine Similarity
    usr_cosine_df = pd.DataFrame(user_recomm_df[input_user_id].sort_values(ascending=False))
    usr_cosine_df.reset_index(level=0, inplace=True)
    usr_cosine_df.columns = ['userId','cosine_sim']
    return usr_cosine_df

In [131]:
def insert_ratings(usr_rtg_dict,pivot_user_based):
    """
    Function to insert new row in pivot_user_based with film ratings created by a new user 
    """
    #loop throgh dictionary with ids of people who rated
    for person_id in usr_rtg_dict:
    #Loop through movies that where rated
        for movie in films_rated.get(person_id):
    #Inset ratings in main DF
              pivot_user_based.loc[person_id,movie] = usr_rtg_dict.get(person_id).get(movie)
#               print("Inserted score for id: ", person_id,movie,usr_rtg_dict.get(person_id).get(movie))
    return pivot_user_based

def user_based_recom(input_dataframe,input_films_rated,user_id):    
    """
    Function to create user based recoomendations.
    """
    pivot_user_based = pd.pivot_table(df, index='userId', columns=['title'], values='rating')
    pivot_user_based = insert_ratings(films_rated,pivot_user_based)
    user_sparse_pivot = sparse.csr_matrix(pivot_user_based.fillna(0))
    user_recommender = pw.cosine_similarity(user_sparse_pivot)
    user_recommender_df = pd.DataFrame(user_recommender, columns=pivot_user_based.index.values,index = pivot_user_based.index.values)
    ## Movie User based Cosine Similarity data frame 
    usr_cosine_df = pd.DataFrame(user_recommender_df[user_id].sort_values(ascending=False))
    usr_cosine_df.reset_index(level=0, inplace=True)
    usr_cosine_df.columns = ['title','cosine_sim']
    ## 4 most similar users
    similar_usr = list(usr_cosine_df['title'][1:5].values)
    ## Comparing reviews with similar users
    similar_usr_df = pivot_user_based.T[[user_id] + similar_usr].fillna(0)
    similar_usr_df['mean_rev'] = similar_usr_df[similar_usr].mean(numeric_only=True,axis=1)
    similar_usr_df.sort_values('mean_rev', ascending=False,inplace = True)
#   Check user rated movies vs similar users ratings
#   display(similar_usr_df[similar_usr_df[user_id]!=0])
    return similar_usr_df[similar_usr_df[user_id]==0].rename({611:'Santosh Cheruku',
                                                              'mean_rev':'Mean_score'},axis=1)

In [132]:
people_who_rated = {611: "Santosh"}
films_rated = movies_rated.to_dict()

# film_name = 'Inception (2010)' 
film_name ='Captain America: The First Avenger (2011)' 
user_id = 611

In [133]:
def generate_recomendations(df,film_name,input_films_rated,top_results=5):
    print("Movie Recommender by B.Kurka:")
    print("User name: " + "Favorite Movie:", film_name+'\n\n')
    print("Films you might enjooy based that you watched", film_name)
    ## Item Rating Based Cosine Similarity
    cos_sim = item_based_recom(df,film_name)
    display(cos_sim[1:top_results+1])
   
    print("***********************************************************************************************\n")
    print("Flims recommended for you:")
    a = user_based_recom(df,input_films_rated,user_id)[0:25]
    
    display(user_based_recom(df,input_films_rated,user_id)[0:5])
#     display()

    return None
    
generate_recomendations(df,film_name,films_rated,5)

Movie Recommender by B.Kurka:
('User name: Favorite Movie:', 'Captain America: The First Avenger (2011)\n\n')
('Films you might enjooy based that you watched', 'Captain America: The First Avenger (2011)')


,title,cosine_sim
1,Thor (2011),0.747897
2,Avengers: Age of Ultron (2015),0.693643
3,Thor: The Dark World (2013),0.668403
4,X-Men: First Class (2011),0.650597
5,X-Men: Days of Future Past (2014),0.650087


***********************************************************************************************

Flims recommended for you:


userId,Santosh Cheruku,298,362,495,295,Mean_score
title,,,,,,
"Shawshank Redemption, The (1994)",0.0,3.5,4.5,5.0,5.0,4.500
Kill Bill: Vol. 1 (2003),0.0,4.0,4.5,5.0,4.5,4.500
Kill Bill: Vol. 2 (2004),0.0,4.0,4.5,5.0,4.5,4.500
"Usual Suspects, The (1995)",0.0,3.5,3.5,4.0,5.0,4.000
"Matrix, The (1999)",0.0,4.0,4.5,5.0,0.0,3.375
